### 1. Importing Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

#!pip install geopy 
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

import requests # library to handle requests
#from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Necessary libraries imported.")

Necessary libraries imported.


### 2. Scrapping the Data and Geocoding Each Neighbourhood

Obtaining the data of different neighbourhood in Bangalore from the Wikipedia page: https://en.wikipedia.org/wiki/List_of_wards_in_Bangalore

Wards are actually local authority areas used for electoral purposes. In Bangalore the civic authority BBMP administers these 198 wards.

In [2]:
WIKI_URL = "https://en.wikipedia.org/wiki/List_of_wards_in_Bangalore"
dfs = pd.read_html(WIKI_URL, header=0)

In [3]:
blr_df = dfs[0]
blr_df = blr_df.drop({'Reservation category','Ward number'}, 1)
blr_df = blr_df.rename(columns={'Ward name':'Neighbourhood','Assembly constituency[2]': 'Assembly constituency','Lok Sabha constituency[3]':'Lok Sabha constituency' })

Adding Latitude and Longitude information to each ward/neighbourhood

In [4]:
#method for geocoding Neighbourhoods in batches
def do_geocode(address, attempt=1, max_attempts=5):
    try:
        geolocator = Nominatim(user_agent="bangalore_explorer")
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        if attempt <= max_attempts:
            return do_geocode(address, attempt=attempt+1)
        raise

In [6]:
latitudes = [] # Initializing the latitude array
longitudes = [] # Initializing the longitude array

for index, row in blr_df.iterrows() :     
    address = row["Neighbourhood"]+", "+row["Lok Sabha constituency"] + ", Bangalore Urban, India" # Formats the place name
    #print(address)
    location = do_geocode(address)
    if bool(location):
        lat = location.latitude
        lng = location.longitude
    
        latitudes.append(lat) # Appending to the list of latitudes
        longitudes.append(lng) # Appending to the list of longitudes
    else:
        latitudes.append('NA') # Appending to the list of latitudes
        longitudes.append('NA') # Appending to the list of longitudes

In [7]:
blr_df['Latitude'] = latitudes
blr_df['Longitude'] = longitudes

In [8]:
#removing rows for which unable to retrieve geocode
blr_df = blr_df[blr_df.Latitude != 'NA']

In [9]:
#resetting table index
blr_df = blr_df.reset_index(drop=True)

In [10]:
blr_df.tail(10)

,Neighbourhood,Assembly constituency,Lok Sabha constituency,Latitude,Longitude
138,Bilekahalli,Bommanahalli,Bangalore South,12.8996,77.6103
139,Hongasandra,Bommanahalli,Bangalore South,12.8975,77.6281
140,Mangammanapalya,Bommanahalli,Bangalore South,12.8958,77.6457
141,Singasandra,Bangalore South,Bangalore Rural,12.8803,77.6548
142,Begur,Bangalore South,Bangalore Rural,12.8634,77.613
143,Arakere,Bommanahalli,Bangalore South,12.8785,77.6042
144,Gottigere,Bangalore South,Bangalore Rural,12.8565,77.5877
145,Konanakunte,Bangalore South,Bangalore Rural,12.8793,77.5698
146,Vasanthapura,Bangalore South,Bangalore Rural,12.8893,77.5596
147,Hemmigepura,Yeshwanthpur,Bangalore North,12.8924,77.4971


In [11]:
#finding colissions for geocoded neighbourhoods
col = 0
explored_lat_lng = []
for lat, lng, neighbourhood in zip(blr_df['Latitude'], blr_df['Longitude'], blr_df['Neighbourhood']):
    if (lat, lng) in explored_lat_lng:
        col = col + 1
        #print(neighbourhood)
    else:
        explored_lat_lng.append((lat, lng))

print("Collisions : ", col)

Collisions :  4


We find only 4 collisions in geocoded neighbourhoods obtained

In [14]:
blr_df.to_csv(r'bangalore_df.csv')

### 3. Mapping the Neighbourhoods

In [15]:
address = 'Bangalore, India'

geolocator = Nominatim(user_agent="blr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of Bangalore, India are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore, India are 12.9791198, 77.5912997.


In [17]:
map_blr = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(blr_df['Latitude'], blr_df['Longitude'], blr_df['Neighbourhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_blr)  
    
map_blr

### 4. Using Foursquare API to get Neighbourhood information

In [18]:
CLIENT_ID = 'MGTGGLSCLUYXDLH2VQ51YTYQIF303HI5PL3NNRECBWDWK3ZV' # your Foursquare ID
CLIENT_SECRET = 'W4RSRNTHLVYWK1NQYCT4QH2FJHBXMCU0OE5PSHECHM1VYJZG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MGTGGLSCLUYXDLH2VQ51YTYQIF303HI5PL3NNRECBWDWK3ZV
CLIENT_SECRET:W4RSRNTHLVYWK1NQYCT4QH2FJHBXMCU0OE5PSHECHM1VYJZG


In [23]:
#creating a function to get the venues
LIMIT = 200
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
#creating a dataframe called blr_venues
blr_venues = getNearbyVenues(names=blr_df['Neighbourhood'],
                                   latitudes=blr_df['Latitude'],
                                   longitudes=blr_df['Longitude']
                                  )
print('All venue information retrieved')

Jakkur
Thanisandra
Byatarayanapura
Kodigehalli
Vidyaranyapura
Doddabommasandra
Kuvempu Nagar
Mallasandra
Bagalagunte
T-Dasarahalli
Jalahalli
Radhakrishna Temple
Sanjay Nagar
Ganga Nagar
Hebbal
Nagavara
HBR Layout
Horamavu
Ramamurthy Nagar
Banaswadi
Kammanahalli
Kacharakanahalli
Kadugondanahalli
Kushal Nagar
Kavalbyrasandra
Manorayanapalya
Gangenahalli
Aramane Nagar
Mathikere
Yeshwanthpur
Chokkasandra
Peenya Industrial Area
Nandini Layout
Malleshwaram
Jayachamarajendra Nagar
Muneshwara Nagar
Lingarajapuram
Benniganahalli
K.R.Puram
Basavanapura
Hoodi
Devasandra
A.Narayanapura
C.V.Raman Nagar
S.K.Garden
Ramaswamy Palya
Jayamahal
Rajamahal
Subramanya Nagar
Nagapura
Mahalakshmipuram
Laggere
Rajagopala Nagar
Hegganahalli
Herohalli
Kottigepalya
Gayathri Nagar
Sarvagna Nagar
Hoysala Nagar
Garudacharpalya
Doddanekkundi
Marathalli
HAL Airport
Jeevanbhima Nagar
Jogupalya
Ulsoor
Bharathi Nagar
Vasanth Nagar
Gandhi Nagar
Subhash Nagar
Okalipuram
Basaveshwara Nagar
Kamakshipalya
Kaveripura
Agrahara 

In [25]:
blr_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Jakkur,13.078474,77.606894,Jus'Trufs Chocolate Shop and Cafe,13.072510,77.603859,Café
1,Jakkur,13.078474,77.606894,MegaJhonson's,13.082980,77.605209,Clothing Store
2,Jakkur,13.078474,77.606894,Naya Daur,13.075757,77.601738,Indian Restaurant
3,Jakkur,13.078474,77.606894,Charlie's Animal Rescue centrE,13.084369,77.608581,Animal Shelter
4,Jakkur,13.078474,77.606894,Ayurdhan Traditional Ayurvedic Healing Center,13.073459,77.613076,Health & Beauty Service


In [27]:
print('There are {} uniques categories.'.format(len(blr_venues['Venue Category'].unique())))

There are 236 uniques categories.


#### One-Hot Encoding

In [28]:
# one hot encoding
blr_onehot = pd.get_dummies(blr_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
blr_onehot['Neighbourhood'] = blr_venues['Neighbourhood'] 

# move neighborhood column to the first column
cols = list(blr_onehot)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Neighbourhood')))
blr_onehot = blr_onehot.loc[:, cols]
blr_onehot.head()

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Terminal,American Restaurant,Andhra Restaurant,Animal Shelter,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Cafeteria,Café,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health & Beauty Service,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Nightlife,Outdoors & Recreation,Paella Restaurant,Paintball Field,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Punjabi Restaurant,Racetrack,Radio Station,Rajasthani Restaurant,Resort,Rest Area,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Szechuan Restaurant,Taco Place,Tea Room,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Toll Booth,Toll Plaza,Toy / Game Store,Track Stadium,Trail,Train Station,Tram Station,Travel & Transport,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Shop,Women's Store,Yoga Studio
0,Jakkur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Jakkur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [29]:
blr_onehot.shape

(4042, 237)

In [30]:
blr_grouped = blr_onehot.groupby('Neighbourhood').mean().reset_index()
blr_grouped.head()

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Terminal,American Restaurant,Andhra Restaurant,Animal Shelter,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Cafeteria,Café,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health & Beauty Service,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Nightlife,Outdoors & Recreation,Paella Restaurant,Paintball Field,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Punjabi Restaurant,Racetrack,Radio Station,Rajasthani Restaurant,Resort,Rest Area,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Szechuan Restaurant,Taco Place,Tea Room,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Toll Booth,Toll Plaza,Toy / Game Store,Track Stadium,Trail,Train Station,Tram Station,Travel & Transport,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Shop,Women's Store,Yoga Studio
0,A.Narayanapura,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.032258,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.064516,0.032258,0.000000,0.032258,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.064516,0.0,0.032258,0.0,0.032258,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.064516,0.0,0.032258,0.0,0.0,0.000000,0.032258,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.096774,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

### 5. Clustering the Neighbourhoods

#### 5.1 Clustering the Neighbourhoods with proximity to Microbreweries.

In [149]:
venue_filter =['Neighbourhood','Brewery']
blr_grouped_filtered=blr_grouped[venue_filter]

In [150]:
blr_grp_clustering = blr_grouped_filtered.drop('Neighbourhood', 1)

In [151]:
kclusters = 3

# Run k-means clustering
kgc = blr_grp_clustering
kmeans = KMeans(n_clusters = kclusters, init = 'k-means++', random_state = 0).fit(kgc)

In [152]:
blr_grouped_filtered.insert(0, 'Cluster Labels', kmeans.labels_)

In [153]:
blr_grouped_filtered.head()

,Cluster Labels,Neighbourhood,Brewery
0,0,A.Narayanapura,0.000000
1,1,Adugodi,0.043478
2,0,Agaram,0.000000
3,0,Agrahara Dasarahalli,0.000000
4,0,Arakere,0.000000


In [154]:
blr_merged = blr_df
blr_merged = blr_merged.join(blr_grouped_filtered.set_index('Neighbourhood'), on='Neighbourhood')
blr_merged.dropna(inplace = True)
blr_merged['Cluster Labels'] = blr_merged['Cluster Labels'].astype(int)
#blr_merged

In [155]:
# Create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(blr_merged['Latitude'], blr_merged['Longitude'], blr_merged['Neighbourhood'], blr_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' (Cluster ' + str(cluster + 1) + ')', parse_html=True)
    map_clusters.add_child(
        folium.features.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7))
       
map_clusters

##### 5.1.1 Examining Clusters

We need to analyze the different clusters

Cluster 1

In [156]:
blr_merged[blr_merged['Cluster Labels']==0]

,Neighbourhood,Assembly constituency,Lok Sabha constituency,Latitude,Longitude,Cluster Labels,Brewery
0,Jakkur,Byatarayanapura,Bangalore North,13.0785,77.6069,0,0.000000
1,Thanisandra,Byatarayanapura,Bangalore North,12.9791,77.5913,0,0.000000
4,Vidyaranyapura,Byatarayanapura,Bangalore North,13.0766,77.5577,0,0.000000
5,Doddabommasandra,Byatarayanapura,Bangalore North,13.065,77.563,0,0.000000
6,Kuvempu Nagar,Byatarayanapura,Bangalore North,13.0732,77.5417,0,0.000000
8,Bagalagunte,Dasarahalli,Bangalore North,13.0563,77.5069,0,0.000000
9,T-Dasarahalli,Dasarahalli,Bangalore North,13.0451,77.5148,0,0.000000
10,Jalahalli,Rajarajeshwarinagar,Bangalore Rural,13.0465,77.5484,0,0.000000
11,Radhakrishna Temple,Hebbal,Bangalore North,13.0337,77.5757,0,0.000000
12,Sanjay Nagar,Hebbal,Bangalore North,12.9579,77.6959,0,0.000000


Cluster 2

In [157]:
blr_merged[blr_merged['Cluster Labels']==1]

,Neighbourhood,Assembly constituency,Lok Sabha constituency,Latitude,Longitude,Cluster Labels,Brewery
2,Byatarayanapura,Byatarayanapura,Bangalore North,13.0621,77.5964,1,0.043478
37,Benniganahalli,CV Raman Nagar,Bangalore Central,12.9949,77.6626,1,0.031250
43,C.V.Raman Nagar,CV Raman Nagar,Bangalore Central,12.9856,77.665,1,0.023810
48,Subramanya Nagar,Malleshwaram,Bangalore North,13.008,77.5572,1,0.013889
64,Jogupalya,Shanthinagar,Bangalore Central,12.9733,77.6338,1,0.014706
68,Gandhi Nagar,Gandhinagar,Bangalore Central,12.9734,77.6112,1,0.020000
69,Subhash Nagar,Gandhinagar,Bangalore Central,12.9734,77.6112,1,0.020000
77,Sampangiram Nagar,Shivajinagar,Bangalore Central,12.9759,77.5906,1,0.014286
78,Domlur,Shanthinagar,Bangalore Central,12.9625,77.6382,1,0.020000
104,Adugodi,B.T.M Layout,Bangalore South,12.9428,77.6104,1,0.043478


Cluster 3

In [158]:
blr_merged[blr_merged['Cluster Labels']==2]

,Neighbourhood,Assembly constituency,Lok Sabha constituency,Latitude,Longitude,Cluster Labels,Brewery
40,Hoodi,Mahadevapura,Bangalore Central,12.9919,77.7162,2,0.083333
103,Lakkasandra,B.T.M Layout,Bangalore South,12.941,77.6032,2,0.058824
127,J.P.Nagar,Jayanagar,Bangalore South,12.9095,77.5967,2,0.076923


#### 5.2 Clustering Neighbourhoods with Potential of opening a Microbrewery. 
We are clustering the neighbourhoods based on similar types of venues like a Microbrewery. These neighbourhoods should have a good potential of setting up a new Microbrewery. 

In [163]:
venue_filter =['Neighbourhood','Pub','Beer Garden','Beer Bar','Food & Drink Shop']
blr_grouped_filtered=blr_grouped[venue_filter]

In [164]:
blr_grp_clustering = blr_grouped_filtered.drop('Neighbourhood', 1)

In [165]:
kclusters = 3

# Run k-means clustering
kgc = blr_grp_clustering
kmeans = KMeans(n_clusters = kclusters, init = 'k-means++', random_state = 0).fit(kgc)

In [166]:
blr_grouped_filtered.insert(0, 'Cluster Labels', kmeans.labels_)

In [167]:
blr_grouped_filtered.head()

,Cluster Labels,Neighbourhood,Pub,Beer Garden,Beer Bar,Food & Drink Shop
0,0,A.Narayanapura,0.0,0.000000,0.0,0.0
1,0,Adugodi,0.0,0.000000,0.0,0.0
2,0,Agaram,0.0,0.000000,0.0,0.0
3,0,Agrahara Dasarahalli,0.0,0.000000,0.0,0.0
4,0,Arakere,0.0,0.030303,0.0,0.0


In [168]:
blr_merged = blr_df
blr_merged = blr_merged.join(blr_grouped_filtered.set_index('Neighbourhood'), on='Neighbourhood')
blr_merged.dropna(inplace = True)
blr_merged['Cluster Labels'] = blr_merged['Cluster Labels'].astype(int)
#blr_merged

In [169]:
# Create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(blr_merged['Latitude'], blr_merged['Longitude'], blr_merged['Neighbourhood'], blr_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' (Cluster ' + str(cluster + 1) + ')', parse_html=True)
    map_clusters.add_child(
        folium.features.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7))
       
map_clusters

##### 5.2.1 Examining Clusters
We need to analyze the different clusters

Cluster 1

In [170]:
blr_merged[blr_merged['Cluster Labels']==0]

,Neighbourhood,Assembly constituency,Lok Sabha constituency,Latitude,Longitude,Cluster Labels,Pub,Beer Garden,Beer Bar,Food & Drink Shop
0,Jakkur,Byatarayanapura,Bangalore North,13.0785,77.6069,0,0.000000,0.000000,0.0,0.000000
1,Thanisandra,Byatarayanapura,Bangalore North,12.9791,77.5913,0,0.000000,0.000000,0.0,0.000000
2,Byatarayanapura,Byatarayanapura,Bangalore North,13.0621,77.5964,0,0.000000,0.000000,0.0,0.000000
4,Vidyaranyapura,Byatarayanapura,Bangalore North,13.0766,77.5577,0,0.000000,0.000000,0.0,0.000000
5,Doddabommasandra,Byatarayanapura,Bangalore North,13.065,77.563,0,0.000000,0.000000,0.0,0.000000
6,Kuvempu Nagar,Byatarayanapura,Bangalore North,13.0732,77.5417,0,0.000000,0.000000,0.0,0.000000
8,Bagalagunte,Dasarahalli,Bangalore North,13.0563,77.5069,0,0.000000,0.000000,0.0,0.000000
9,T-Dasarahalli,Dasarahalli,Bangalore North,13.0451,77.5148,0,0.000000,0.000000,0.0,0.000000
10,Jalahalli,Rajarajeshwarinagar,Bangalore Rural,13.0465,77.5484,0,0.000000,0.000000,0.0,0.000000
11,Radhakrishna Temple,Hebbal,Bangalore North,13.0337,77.5757,0,0.000000,0.000000,0.0,0.000000


Cluster 2

In [171]:
blr_merged[blr_merged['Cluster Labels']==1]

,Neighbourhood,Assembly constituency,Lok Sabha constituency,Latitude,Longitude,Cluster Labels,Pub,Beer Garden,Beer Bar,Food & Drink Shop
64,Jogupalya,Shanthinagar,Bangalore Central,12.9733,77.6338,1,0.088235,0.0,0.0,0.014706
78,Domlur,Shanthinagar,Bangalore Central,12.9625,77.6382,1,0.050000,0.0,0.0,0.020000


Cluster 3

In [172]:
blr_merged[blr_merged['Cluster Labels']==2]

,Neighbourhood,Assembly constituency,Lok Sabha constituency,Latitude,Longitude,Cluster Labels,Pub,Beer Garden,Beer Bar,Food & Drink Shop
46,Jayamahal,Shivajinagar,Bangalore Central,12.9988,77.5976,2,0.038462,0.0,0.000000,0.000000
48,Subramanya Nagar,Malleshwaram,Bangalore North,13.008,77.5572,2,0.013889,0.0,0.013889,0.000000
63,Jeevanbhima Nagar,CV Raman Nagar,Bangalore Central,12.9649,77.6512,2,0.031250,0.0,0.000000,0.000000
65,Ulsoor,Shivajinagar,Bangalore Central,12.9779,77.6247,2,0.019608,0.0,0.000000,0.000000
67,Vasanth Nagar,Shivajinagar,Bangalore Central,12.9887,77.5852,2,0.016393,0.0,0.000000,0.016393
68,Gandhi Nagar,Gandhinagar,Bangalore Central,12.9734,77.6112,2,0.040000,0.0,0.000000,0.000000
69,Subhash Nagar,Gandhinagar,Bangalore Central,12.9734,77.6112,2,0.040000,0.0,0.000000,0.000000
77,Sampangiram Nagar,Shivajinagar,Bangalore Central,12.9759,77.5906,2,0.014286,0.0,0.000000,0.000000
83,Shanthi Nagar,Shanthinagar,Bangalore Central,12.9583,77.6026,2,0.030303,0.0,0.000000,0.000000
107,Bellandur,Mahadevapura,Bangalore Central,12.9734,77.6112,2,0.040000,0.0,0.000000,0.000000
